## Install YOLO-NAS


In [1]:
!pip install -q git+https://github.com/Deci-AI/super-gradients.git@stable
!pip install -q roboflow
!pip install -q supervision

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
ucxx 0.39.1 requires libucx>=1.15.0, which is not installed.
aiobotocore 2.13.2 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
albucore 0.0.13 requires numpy<2,>=1.24.4, but you have numpy 1.23.0 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [3]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'

In [4]:
from super_gradients.training import models

model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)

The console stream is logged into /root/sg_logs/console.log


[2024-09-23 16:07:54] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-09-23 16:08:10] INFO - utils.py - NumExpr defaulting to 4 threads.


[WARNING]No module named 'pycocotools'


[2024-09-23 16:08:14] INFO - env_sanity_check.py - Library check is not supported when super_gradients installed through "git+https://github.com/..." command
[2024-09-23 16:08:15] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-09-23 16:08:15] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_l

In [5]:
%cd {HOME}

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="OX5LGF0zNUWp6nV9jsTi")
project = rf.workspace("object-detection-1y7d0").project("military-aircraft-iaddd")
version = project.version(7)
dataset = version.download("yolov5")

/kaggle/working


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to military-aircraft-7 in yolov5pytorch::  76%|███████▌  | 6460/8511 [00:00<00:00, 10144.59it/s]

In [6]:
LOCATION = dataset.location
print("location:", LOCATION)
CLASSES = sorted(project.classes.keys())
print("classes:", CLASSES)

In [7]:
MODEL_ARCH = 'yolo_nas_l'
BATCH_SIZE = 8
MAX_EPOCHS = 25
CHECKPOINT_DIR = f'{HOME}/checkpoints'
EXPERIMENT_NAME = project.name.lower().replace(" ", "_")

In [8]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [9]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}

In [10]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

# test_data = coco_detection_yolo_format_val(
#     dataset_params={
#         'data_dir': dataset_params['data_dir'],
#         'images_dir': dataset_params['test_images_dir'],
#         'labels_dir': dataset_params['test_labels_dir'],
#         'classes': dataset_params['classes']
#     },
#     dataloader_params={
#         'batch_size': BATCH_SIZE,
#         'num_workers': 2
#     }
# )

[2024-09-23 16:08:36] WARNING - yolo_format_detection.py - 127 images are note associated to any label file
[2024-09-23 16:08:36] WARNING - yolo_format_detection.py - As a consequence, 3402/3529 images and 3402/3402 label files will be used.
[2024-09-23 16:08:36] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 3402/3402 [00:00<00:00, 6066.04it/s]
[2024-09-23 16:08:37] WARNING - yolo_format_detection.py - 439 images are note associated to any label file
[2024-09-23 16:08:37] WARNING - yolo_format_detection.py - As a consequence, 566/1005 images and 566/566 label files will be used.
[2024-09-23 16:08:37] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.


**NOTE:** Let's inspect our dataset

In [11]:
train_data.dataset.transforms

### Instantiate the model

In [12]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

[2024-09-23 16:08:38] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-09-23 16:08:38] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-09-23 16:08:38] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


### Define metrics and training parameters

In [13]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

### Train the model

In [14]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

[2024-09-23 16:08:39] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20240923_160839_191420`
[2024-09-23 16:08:39] INFO - sg_trainer.py - Checkpoints directory: /kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420
[2024-09-23 16:08:39] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}


The console stream is now moved to /kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/console_Sep23_16_08_39.txt


Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
initialize_param_groups and update_param_groups usages are deprecated since 3.4.0, will be removed in 3.5.0 and have no effect. 
 Assign different learning rates by passing a mapping of layer name prefixes to lr values through initial_lr training hyperparameter (i.e initial_lr={'backbone': 0.01, 'default':0.1})
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
[2024-09-23 16:08:41] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (2 available on the machine)
    - Full dataset size:            3402       (len(train_set))
    - Batch size per GPU:           8          (batch_size)
    - Batch Accumula

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 2.0713
│   ├── Ppyoloeloss/loss_iou = 0.5412
│   ├── Ppyoloeloss/loss_dfl = 0.6738
│   └── Ppyoloeloss/loss = 3.2863
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.2342
    ├── Ppyoloeloss/loss_iou = 0.3717
    ├── Ppyoloeloss/loss_dfl = 0.5115
    ├── Ppyoloeloss/loss = 2.1174
    ├── Precision@0.50 = 0.1694
    ├── Recall@0.50 = 0.7897
    ├── Map@0.50 = 0.5467
    ├── F1@0.50 = 0.2789
    └── Best_score_threshold = 0.28



Validating epoch 1: 100%|██████████| 71/71 [00:26<00:00,  2.71it/s]
[2024-09-23 16:18:00] INFO - base_sg_logger.py - Checkpoint saved in /kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/ckpt_best.pth
[2024-09-23 16:18:00] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.8207380771636963


...

Validating epoch 23: 100%|██████████| 71/71 [00:26<00:00,  2.71it/s]
[2024-09-23 18:04:54] INFO - base_sg_logger.py - Checkpoint saved in /kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/ckpt_best.pth
[2024-09-23 18:04:54] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.9075803756713867


SUMMARY OF EPOCH 23
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.8517
│   │   ├── Epoch N-1      = 0.8717 (↘ -0.02)
│   │   └── Best until now = 0.8681 (↘ -0.0164)
│   ├── Ppyoloeloss/loss_iou = 0.3858
│   │   ├── Epoch N-1      = 0.3889 (↘ -0.0031)
│   │   └── Best until now = 0.3889 (↘ -0.0031)
│   ├── Ppyoloeloss/loss_dfl = 0.5249
│   │   ├── Epoch N-1      = 0.5277 (↘ -0.0029)
│   │   └── Best until now = 0.5272 (↘ -0.0023)
│   └── Ppyoloeloss/loss = 1.7624
│       ├── Epoch N-1      = 1.7884 (↘ -0.026)
│       └── Best until now = 1.7878 (↘ -0.0254)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.7508
    │   ├── Epoch N-1      = 0.7401 (↗ 0.0107)
    │   └── Best until now = 0.7401 (↗ 0.0107)
    ├── Ppyoloeloss/loss_iou = 0.2705
    │   ├── Epoch N-1      = 0.2728 (↘ -0.0023)
    │   └── Best until now = 0.2728 (↘ -0.0023)
    ├── Ppyoloeloss/loss_dfl = 0.4211
    │   ├── Epoch N-1      = 0.4207 (↗ 0.0004)
    │   └── Best until now = 0.4207 (↗ 0.0004)
    ├── Ppyoloeloss/loss = 1

Validating epoch 24: 100%|██████████| 71/71 [00:26<00:00,  2.71it/s]


SUMMARY OF EPOCH 24
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.8502
│   │   ├── Epoch N-1      = 0.8517 (↘ -0.0016)
│   │   └── Best until now = 0.8517 (↘ -0.0016)
│   ├── Ppyoloeloss/loss_iou = 0.38
│   │   ├── Epoch N-1      = 0.3858 (↘ -0.0058)
│   │   └── Best until now = 0.3858 (↘ -0.0058)
│   ├── Ppyoloeloss/loss_dfl = 0.5219
│   │   ├── Epoch N-1      = 0.5249 (↘ -0.003)
│   │   └── Best until now = 0.5249 (↘ -0.003)
│   └── Ppyoloeloss/loss = 1.7521
│       ├── Epoch N-1      = 1.7624 (↘ -0.0104)
│       └── Best until now = 1.7624 (↘ -0.0104)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.7326
    │   ├── Epoch N-1      = 0.7508 (↘ -0.0183)
    │   └── Best until now = 0.7401 (↘ -0.0076)
    ├── Ppyoloeloss/loss_iou = 0.2827
    │   ├── Epoch N-1      = 0.2705 (↗ 0.0122)
    │   └── Best until now = 0.2705 (↗ 0.0122)
    ├── Ppyoloeloss/loss_dfl = 0.4264
    │   ├── Epoch N-1      = 0.4211 (↗ 0.0053)
    │   └── Best until now = 0.4207 (↗ 0.0057)
    ├── Ppyoloeloss/loss = 1.

[2024-09-23 18:09:51] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Validating epoch 25: 100%|██████████| 71/71 [00:26

### Analyze training metrics


In [15]:
# %load_ext tensorboard
# %tensorboard --logdir {CHECKPOINT_DIR}/{EXPERIMENT_NAME}

## Zip and download fine-tuned model

In [16]:
!zip -r yolo_nas.zip {CHECKPOINT_DIR}/{EXPERIMENT_NAME}

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


  adding: kaggle/working/checkpoints/military_aircraft/ (stored 0%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/ (stored 0%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/logs_Sep23_16_08_39.txt (deflated 78%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/console_Sep23_16_08_39.txt (deflated 96%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/ckpt_latest.pth (deflated 7%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/events.out.tfevents.1727115019.c0c463887d6f.24.1 (deflated 9%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/average_model.pth (deflated 7%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/ckpt_best.pth (deflated 7%)
  adding: kaggle/working/checkpoints/military_aircraft/RUN_20240923_160839_191420/experiment_logs_Sep23_16_08_3

In [17]:
# if you experience 'NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968' error, run code below 👇

# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

## Load trained model

In [18]:
# best_model = models.get(
#     MODEL_ARCH,
#     num_classes=len(dataset_params['classes']),
#     checkpoint_path=f"/kaggle/input/yolo-nas/pytorch/default/1/ckpt_best.pth"
# ).to(DEVICE)

In [19]:
# # Import the YouTubeVideo class from IPython.display
# from IPython.display import YouTubeVideo

# # Define the YouTube video ID
# video_id = 'l-oo9DJfH_k'  # Replace YOUR_VIDEO_ID with the actual video ID

# # Create a YouTubeVideo object with the specified video ID
# video = YouTubeVideo(video_id)

# # Display the video
# display(video)

In [20]:
# %%capture

# # Define the URL of the YouTube video
# video_url = f'https://www.youtube.com/watch?v={video_id}'

# # Download the video in mp4 format
# !pip install -U "git+https://github.com/ytdl-org/youtube-dl.git"
# !python -m youtube_dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' "$video_url"

# # Print a success message
# print('Video downloaded successfully')


In [21]:
input_video_path = "/kaggle/working/Battlefield 2042 Trailer - F-35 fighter jet scene-l-oo9DJfH_k.mp4"
output_video_path = "detections.mp4"

In [22]:
# import torch
# device = 'cuda' if torch.cuda.is_available() else "cpu"

In [23]:
# best_model.to(device).predict(input_video_path).save(output_video_path)

## Evaluate trained model

In [24]:
# trainer.test(
#     model=best_model,
#     test_loader=test_data,
#     test_metrics_list=DetectionMetrics_050(
#         score_thres=0.1,
#         top_k_predictions=300,
#         num_cls=len(dataset_params['classes']),
#         normalize_targets=True,
#         post_prediction_callback=PPYoloEPostPredictionCallback(
#             score_threshold=0.01,
#             nms_top_k=1000,
#             max_predictions=300,
#             nms_threshold=0.7
#         )
#     )
# )

### Calculate confusion matrix

In [25]:
# !pip install onemetric

In [26]:
# import os

# import numpy as np

# from onemetric.cv.object_detection import ConfusionMatrix

# keys = list(ds.images.keys())

# annotation_batches, prediction_batches = [], []

# for key in keys:
#     annotation=ds.annotations[key]
#     annotation_batch = np.column_stack((
#         annotation.xyxy,
#         annotation.class_id
#     ))
#     annotation_batches.append(annotation_batch)

#     prediction=predictions[key]
#     prediction_batch = np.column_stack((
#         prediction.xyxy,
#         prediction.class_id,
#         prediction.confidence
#     ))
#     prediction_batches.append(prediction_batch)

# confusion_matrix = ConfusionMatrix.from_detections(
#     true_batches=annotation_batches,
#     detection_batches=prediction_batches,
#     num_classes=len(ds.classes),
#     conf_threshold=CONFIDENCE_TRESHOLD
# )

# confusion_matrix.plot(os.path.join(HOME, "confusion_matrix.png"), class_names=ds.classes)